In [1]:
import os
import sys

# Save current working directory
cwd = os.getcwd()

# Change to parent directory
parent_dir = os.path.abspath(os.path.join(cwd, '..'))
os.chdir(parent_dir)

# Temporarily add parent directory to sys.path
sys.path.insert(0, parent_dir)
import framework
sys.path.pop(0)

# Return to original directory
os.chdir(cwd)

In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models import vit_b_32, ViT_B_32_Weights
from torch import nn, optim
from tqdm import tqdm

weights = ViT_B_32_Weights.DEFAULT
# Transform matching ImageNet-trained model
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    weights.transforms()
])

model = vit_b_32()
model.heads = nn.Linear(in_features = 768, out_features = 100, bias = True)

device = "cuda"
state_dict = torch.load('model_weights.pth', map_location=device)
model.load_state_dict(state_dict)

<All keys matched successfully>

In [3]:
from tqdm import tqdm

test_data = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_data, batch_size=64)

from framework.criterion import classification_accuracy

all_images = []
all_labels = []

for images, labels in tqdm(test_loader):
    all_images.append(images)
    all_labels.append(labels)

X_tensor = torch.cat(all_images).to(device)
y_true = torch.cat(all_labels).to(device)

classification_accuracy(model, X_tensor, y_true, device)

Files already downloaded and verified


100%|██████████| 157/157 [00:17<00:00,  8.96it/s]
c:\Repositories\seu-injection-framework\.venv\Lib\site-packages\torch\nn\modules\activation.py:1196: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  return torch._native_multi_head_attention(


0.0348

In [4]:
from framework.attack import Injector

inj = Injector(model, X_tensor, y_true, classification_accuracy, device=device)

Testing a forward pass on cuda...
Basline Criterion Score: 0.0348


In [5]:
layer_name = "heads.bias"

results = inj.run_stochastic_seu(1,1/20,layer_name)

results

Testing Layer:  heads.bias


100it [01:52,  1.13s/it]


{'tensor_location': [(28,), (29,), (50,), (79,), (80,), (85,)],
 'criterion_score': [0.01, 0.0348, 0.01, 0.01, 0.01, 0.0348],
 'layer_name': ['heads.bias',
  'heads.bias',
  'heads.bias',
  'heads.bias',
  'heads.bias',
  'heads.bias'],
 'value_before': [0.0047611212,
  -0.010861064,
  0.022124333,
  0.022607008,
  0.008148461,
  -0.002507337],
 'value_after': [1.6201256033114394e+36,
  -3.6958286697997516e+36,
  7.528520504303744e+36,
  7.692766189623515e+36,
  2.7727774763881643e+36,
  -8.532025361447367e+35]}

In [6]:
import pandas as pd

pd.DataFrame(results)

,tensor_location,criterion_score,layer_name,value_before,value_after
0,"(28,)",0.0100,heads.bias,0.004761,1.620126e+36
1,"(29,)",0.0348,heads.bias,-0.010861,-3.695829e+36
2,"(50,)",0.0100,heads.bias,0.022124,7.528521e+36
3,"(79,)",0.0100,heads.bias,0.022607,7.692766e+36
4,"(80,)",0.0100,heads.bias,0.008148,2.772777e+36
5,"(85,)",0.0348,heads.bias,-0.002507,-8.532025e+35
